In [1]:
import pandas as pd

# Define the original weights for each 'CID' value in a dictionary
original_weights = {
    1: 1/8,
    2: 1/8,
    3: 1/4,
    4: 1/2
}

# Define the list of columns or variables you want to calculate weighted averages for (blood variables)
blood_variables = [
    'ecid_hba1c_percent',
    'ecid_ldl',
    'ecid_hdl',
    'ecid_trig'
]  # Add or remove columns as needed

# Load your Excel data
PATH = '/Users/user/Downloads/DellaCorte/XSLX/Merged.xlsx'
sheet_name = 'Merged_All_Data'
df = pd.read_excel(PATH, sheet_name)

# Create the 'blood_df' DataFrame
blood_df = df[['participant_id', 'CID'] + blood_variables]

# Get a list of unique 'participant_id' values
unique_participants = blood_df['participant_id'].unique()

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['participant_id'] + ['Weighted Average for ' + col for col in blood_variables])

# Iterate through each 'participant_id'
for target_participant in unique_participants:
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning for each participant
    blood_df_copy = blood_df.copy()
    
    # Check if the current 'participant_id' is present in the DataFrame
    if target_participant in blood_df_copy['participant_id'].unique():
        # Get the 'CID' values associated with the 'participant_id'
        valid_cids = blood_df_copy[(blood_df_copy['participant_id'] == target_participant) & blood_df_copy['CID'].notna()]['CID'].unique()
        
        # Initialize a list to store weighted averages for each column of interest
        weighted_averages = [target_participant]
        
        for column in blood_variables:
            # Calculate the weighted average for the current column and 'participant_id'
            weighted_average = 0
            
            # Check if there are valid 'CID' values
            if len(valid_cids) > 0:
                # Select only data where 'column' is not NaN
                valid_data = blood_df_copy[
                    (blood_df_copy['participant_id'] == target_participant) & 
                    blood_df_copy[column].notna()
                ]
                
                if not valid_data.empty:
                    # Calculate the sum of original weights for the available 'CID' values
                    sum_original_weights = sum(original_weights[cid] for cid in valid_data['CID'])
                    
                    # Calculate the rescaled weights to ensure they add up to 1
                    valid_data['rescaled_weight'] = valid_data['CID'].map(original_weights) / sum_original_weights
                    
                    # Calculate the weighted average for the current column and 'participant_id'
                    weighted_average = (valid_data[column] * valid_data['rescaled_weight']).sum()
            
            weighted_averages.append(weighted_average)
        
        # Append the results to the result_df
        result_df = result_df.append(pd.Series(weighted_averages, index=result_df.columns), ignore_index=True)
    else:
        print(f"Target Participant {target_participant} is not present in the DataFrame.")

# Print the resulting DataFrame
print(result_df)

# Save the result as an Excel (.xlsx) file
result_df.to_excel('blood_weighted_averages.xlsx', index=False)


/var/folders/td/8px6smg12fxf4nw_rf4snx840000gn/T/ipykernel_72766/3507298286.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['rescaled_weight'] = valid_data['CID'].map(original_weights) / sum_original_weights
/var/folders/td/8px6smg12fxf4nw_rf4snx840000gn/T/ipykernel_72766/3507298286.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['rescaled_weight'] = valid_data['CID'].map(original_weights) / sum_original_weights
/var/folders/td/8px6smg12fxf4nw_rf4snx840000gn/T/ipykernel_72766

AttributeError: 'DataFrame' object has no attribute 'append'